In [3]:
import pandas as pd
import numpy as np

In [42]:
np.random.choice(['x', 'y'], size=20)

array(['x', 'x', 'x', 'x', 'y', 'x', 'y', 'y', 'x', 'x', 'y', 'x', 'y',
       'x', 'x', 'x', 'y', 'y', 'x', 'x'], dtype='<U1')

#### Dummy Model Results

In [130]:
n = 100
series_list = []
headers = ['real'] + [f'model_{i}' for i in range(0, 10)]
data_hashmap = {}
for header in headers:
    data_hashmap[header] = list(np.random.choice(['x', 'y'], size=n))

df = pd.DataFrame.from_dict(data_hashmap)
df.head(20)


,real,model_0,model_1,model_2,model_3,model_4,model_5,model_6,model_7,model_8,model_9
0,y,x,x,x,x,x,x,x,x,x,y
1,y,x,x,y,y,y,y,y,x,x,x
2,y,y,x,y,x,x,y,y,x,x,x
3,x,x,x,y,x,y,x,y,x,y,y
4,y,y,y,x,x,y,x,x,x,y,y
5,y,y,y,x,y,y,x,y,x,x,x
6,x,x,y,y,y,y,y,x,y,y,x
7,y,y,x,y,y,y,x,y,x,y,x
8,y,x,y,x,x,y,x,y,x,y,x
9,y,y,x,y,x,x,y,x,y,y,x


In [131]:
for i, _ in enumerate(headers[1:]):
    df[f'accuracy_{i}'] = np.where(df['real'] == df[f'model_{i}'], 1, 0)

df[['real', 'model_0', 'accuracy_0']].head(10)

,real,model_0,accuracy_0
0,y,x,0
1,y,x,0
2,y,y,1
3,x,x,1
4,y,y,1
5,y,y,1
6,x,x,1
7,y,y,1
8,y,x,0
9,y,y,1


#### create Xs and Ys count for model

In [132]:
def create_value_counts_columns(df, col, values):
    counts = df[[col]].value_counts()
    for v in values:
        count_v = counts.get(v)
        df[f'{col}_count_{v}'] = count_v
    return df



#### Computing Precion, Recall, F1

In [144]:
def compute_precision_recall_f1(df, model):
    df = df.copy()
    df = create_value_counts_columns(df, 'real', ['x', 'y'])
    df = create_value_counts_columns(df, model, ['x', 'y'])
    df[f'{model}_x_accuracy'] = np.where((df['accuracy_0'] == 1) & (df[f'{model}'] == 'x'), 1, 0)
    df[f'{model}_y_accuracy'] = np.where((df['accuracy_0'] == 1) & (df[f'{model}'] == 'y'), 1, 0)

    sum_x_accuracy = int(df[[f'{model}_x_accuracy']].sum())
    sum_y_accuracy = int(df[[f'{model}_y_accuracy']].sum())

    df[f'{model}_x_accurate'] = sum_x_accuracy
    df[f'{model}_x_inaccurate'] = df[f'{model}_count_x'] - df[f'{model}_x_accurate']
    df[f'{model}_y_accurate'] = sum_y_accuracy
    df[f'{model}_y_inaccurate'] = df[f'{model}_count_y'] - df[f'{model}_y_accurate']
    df[f'{model}_observations_x'] = df[f'{model}_x_accurate'] + df[f'{model}_y_inaccurate']
    df[f'{model}_observations_y'] = df[f'{model}_y_accurate'] + df[f'{model}_x_inaccurate']

    precision_x, precision_y = f'precision_x', f'precision_y'
    recall_x, recall_y = f'recall_x', f'recall_y'
    f1_score_x, f1_score_y = 'f1_score_x', 'f1_score_y'
    df[precision_x] = df[f'{model}_x_accurate'] / df[f'{model}_observations_x']
    df[recall_x] = df[f'{model}_x_accurate'] / df[f'{model}_count_x']
    df[precision_y] = df[f'{model}_y_accurate'] / df[f'{model}_observations_y']
    df[recall_y] = df[f'{model}_y_accurate'] / df[f'{model}_count_y']

    df[f1_score_x] = (df[precision_x] * df[recall_x] * 2) / (df[precision_x] + df[recall_x])
    df[f1_score_y] = (df[precision_y] * df[recall_y] * 2) / (df[precision_y] + df[recall_y])

    df['precision(x+y)'] = (df[precision_x] + df[precision_y]) / 2
    df['recall(x+y)'] = (df[recall_x] + df[recall_y]) / 2
    df['f1_score(x+y)'] = (df[f1_score_x] + df[f1_score_y]) / 2

    return df

In [145]:
select_cols = ['precision_x', 'recall_x', 'f1_score_x', 'precision_y', 'recall_y', 'f1_score_y', 'precision(x+y)', 'recall(x+y)', 'f1_score(x+y)']

In [150]:
precision_and_recall = compute_precision_recall_f1(df, 'model_0')
precision_and_recall[select_cols].head(1)

,precision_x,recall_x,f1_score_x,precision_y,recall_y,f1_score_y,precision(x+y),recall(x+y),f1_score(x+y)
0,0.481481,0.577778,0.525253,0.586957,0.490909,0.534653,0.534219,0.534343,0.529953


In [155]:
precision_and_recall = compute_precision_recall_f1(df, 'model_1')
precision_and_recall[select_cols].head(1)

,precision_x,recall_x,f1_score_x,precision_y,recall_y,f1_score_y,precision(x+y),recall(x+y),f1_score(x+y)
0,0.537037,0.568627,0.552381,0.521739,0.489796,0.505263,0.529388,0.529212,0.528822


In [152]:
precision_and_recall = compute_precision_recall_f1(df, 'model_2')
precision_and_recall[select_cols].head(1)

,precision_x,recall_x,f1_score_x,precision_y,recall_y,f1_score_y,precision(x+y),recall(x+y),f1_score(x+y)
0,0.54902,0.538462,0.543689,0.510204,0.520833,0.515464,0.529612,0.529647,0.529577


In [154]:
precision_and_recall = compute_precision_recall_f1(df, 'model_3')
precision_and_recall[select_cols].head(1)

,precision_x,recall_x,f1_score_x,precision_y,recall_y,f1_score_y,precision(x+y),recall(x+y),f1_score(x+y)
0,0.520833,0.510204,0.515464,0.538462,0.54902,0.543689,0.529647,0.529612,0.529577
